In [10]:
# Plan 
# 1) Import the libraries
# 2) Read all the files in the bucket
# 3) Create a list of the files
# 4) Extract the actual date
# 5) Extract the actual hour less 1 hour
# 6) Create String with date + 'T' + actual hour in 24 format ex 2021-10-11T14 and call it file_not_import / UTC timezone
# 7) Prepare the Storage 
# 8) Prepare the BigQuery Connection
# 9) Insert all the files that the string is not file_not_import and are a valid .csv.gz file
# 9.1) After Import move the file to Imported folder

In [11]:
# 1) Import the libraries
import sys
import requests
import json
import pandas as pd
import math
from google.cloud import bigquery
from google.cloud import storage
import time
from datetime import date
from datetime import datetime
from google.cloud import storage

In [12]:
# 2) Read all the files in the bucket
client = storage.Client()
BUCKET_NAME = 'tgm-adjust'
bucket = client.get_bucket(BUCKET_NAME)

In [13]:
# 3) Create a list of the files
files = bucket.list_blobs()

In [14]:
# 4) Extract the actual date
today = str(date.today())

In [15]:
# 5) Extract the actual hour less 1 hour
hour = str(datetime.now().hour - 1)

In [16]:
# 6) Create String with date + 'T' + actual hour in 24 format ex 2021-10-11T14 and call it file_not_import
file_not_import = (today+'T'+hour)

In [17]:
# 7) Prepare the Storage 

def move_blob(bucket_name, blob_name, destination_bucket_name, destination_blob_name):
    """Moves a blob from one bucket to another with a new name."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"
    # The ID of your GCS object
    # blob_name = "your-object-name"
    # The ID of the bucket to move the object to
    # destination_bucket_name = "destination-bucket-name"
    # The ID of your new GCS object (optional)
    # destination_blob_name = "destination-object-name"

    storage_client = storage.Client()

    source_bucket = storage_client.bucket(bucket_name)
    source_blob = source_bucket.blob(blob_name)
    destination_bucket = storage_client.bucket(destination_bucket_name)

    blob_copy = source_bucket.copy_blob(
        source_blob, destination_bucket, destination_blob_name
    )
    source_bucket.delete_blob(blob_name)

    print(
        "Blob {} in bucket {} moved to blob {} in bucket {}.".format(
            source_blob.name,
            source_bucket.name,
            blob_copy.name,
            destination_bucket.name,
        )
    )

In [18]:
# 8) Prepare the BigQuery Connection

client = bigquery.Client()
table_id = "tgm-data-325414.adjust.raw_data"

job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("app_id","STRING"),
        bigquery.SchemaField("app_name","STRING"),
        bigquery.SchemaField("app_name_dashboard","STRING"),
        bigquery.SchemaField("app_version","STRING"),
        bigquery.SchemaField("app_version_raw","STRING"),
        bigquery.SchemaField("app_version_short","STRING"),
        bigquery.SchemaField("store","STRING"),
        bigquery.SchemaField("tracker","STRING"),
        bigquery.SchemaField("tracker_name","STRING"),
        bigquery.SchemaField("first_tracker","STRING"),
        bigquery.SchemaField("first_tracker_name","STRING"),
        bigquery.SchemaField("last_tracker","STRING"),
        bigquery.SchemaField("last_tracker_name","STRING"),
        bigquery.SchemaField("outdated_tracker","STRING"),
        bigquery.SchemaField("network_name","STRING"),
        bigquery.SchemaField("campaign_name","STRING"),
        bigquery.SchemaField("adgroup_name","STRING"),
        bigquery.SchemaField("creative_name","STRING"),
        bigquery.SchemaField("impression_based","STRING"),
        bigquery.SchemaField("is_organic","STRING"),
        bigquery.SchemaField("is_s2s","STRING"),
        bigquery.SchemaField("is_s2s_engagement_based","STRING"),
        bigquery.SchemaField("rejection_reason","STRING"),
        bigquery.SchemaField("click_referer","STRING"),
        bigquery.SchemaField("activity_kind","STRING"),
        bigquery.SchemaField("click_time","STRING"),
        bigquery.SchemaField("click_time_hour","STRING"),
        bigquery.SchemaField("impression_time","STRING"),
        bigquery.SchemaField("impression_time_hour","STRING"),
        bigquery.SchemaField("conversion_duration","STRING"),
        bigquery.SchemaField("engagement_time","STRING"),
        bigquery.SchemaField("engagement_time_hour","STRING"),
        bigquery.SchemaField("installed_at","STRING"),
        bigquery.SchemaField("installed_at_hour","STRING"),
        bigquery.SchemaField("install_finish_time","STRING"),
        bigquery.SchemaField("install_begin_time","STRING"),
        bigquery.SchemaField("referral_time","STRING"),
        bigquery.SchemaField("created_at","STRING"),
        bigquery.SchemaField("created_at_milli","STRING"),
        bigquery.SchemaField("created_at_hour","STRING"),
        bigquery.SchemaField("received_at","STRING"),
        bigquery.SchemaField("reattributed_at","STRING"),
        bigquery.SchemaField("reattributed_at_hour","STRING"),
        bigquery.SchemaField("attribution_updated_at","STRING"),
        bigquery.SchemaField("time_to_uninstall","STRING"),
        bigquery.SchemaField("time_to_reinstall","STRING"),
        bigquery.SchemaField("uninstalled_at","STRING"),
        bigquery.SchemaField("reinstalled_at","STRING"),
        bigquery.SchemaField("last_session_time","STRING"),
        bigquery.SchemaField("connection_type","STRING"),
        bigquery.SchemaField("cpu_type","STRING"),
        bigquery.SchemaField("hardware_name","STRING"),
        bigquery.SchemaField("network_type","STRING"),
        bigquery.SchemaField("device_manufacturer","STRING"),
        bigquery.SchemaField("proxy_ip_address","STRING"),
        bigquery.SchemaField("ad_revenue_network","STRING"),
        bigquery.SchemaField("ad_revenue_unit","STRING"),
        bigquery.SchemaField("ad_revenue_placement","STRING"),
        bigquery.SchemaField("ad_mediation_platform","STRING"),
        bigquery.SchemaField("ad_revenue_payload","STRING"),
        bigquery.SchemaField("ad_impressions_count","STRING"),
        bigquery.SchemaField("dcp_xxx","STRING"),
        bigquery.SchemaField("is_deeplink_click","STRING"),
        bigquery.SchemaField("san_engagement_times","STRING"),
        bigquery.SchemaField("device_atlas_id","STRING"),
        bigquery.SchemaField("click_attribution_window","STRING"),
        bigquery.SchemaField("impression_attribution_window","STRING"),
        bigquery.SchemaField("probmatching_attribution_window","STRING"),
        bigquery.SchemaField("reattribution_attribution_window","STRING"),
        bigquery.SchemaField("inactive_user_definition","STRING"),
        bigquery.SchemaField("attribution_ttl","STRING"),
        bigquery.SchemaField("reattribution_attribution_ttl","STRING"),
        bigquery.SchemaField("reattribution_fallback_type","STRING"),
        bigquery.SchemaField("last_fallback_time","STRING"),
        bigquery.SchemaField("attribution_expires_at","STRING"),
        bigquery.SchemaField("callback_ttl","STRING"),
        bigquery.SchemaField("within_callback_ttl","STRING"),
        bigquery.SchemaField("adid","STRING"),
        bigquery.SchemaField("idfa","STRING"),
        bigquery.SchemaField("android_id","STRING"),
        bigquery.SchemaField("android_id_md5","STRING"),
        bigquery.SchemaField("mac_sha1","STRING"),
        bigquery.SchemaField("mac_md5","STRING"),
        bigquery.SchemaField("idfa_android_id","STRING"),
        bigquery.SchemaField("idfa_gps_adid","STRING"),
        bigquery.SchemaField("idfa_gps_adid_fire_adid","STRING"),
        bigquery.SchemaField("idfa_md5","STRING"),
        bigquery.SchemaField("idfa_md5_hex","STRING"),
        bigquery.SchemaField("idfa_upper","STRING"),
        bigquery.SchemaField("idfv","STRING"),
        bigquery.SchemaField("gps_adid","STRING"),
        bigquery.SchemaField("gps_adid_md5","STRING"),
        bigquery.SchemaField("oaid","STRING"),
        bigquery.SchemaField("oaid_md5","STRING"),
        bigquery.SchemaField("external_device_id_md5","STRING"),
        bigquery.SchemaField("web_uuid","STRING"),
        bigquery.SchemaField("win_udid","STRING"),
        bigquery.SchemaField("win_hwid","STRING"),
        bigquery.SchemaField("win_naid","STRING"),
        bigquery.SchemaField("win_adid","STRING"),
        bigquery.SchemaField("fire_adid","STRING"),
        bigquery.SchemaField("match_type","STRING"),
        bigquery.SchemaField("reftag","STRING"),
        bigquery.SchemaField("reftags","STRING"),
        bigquery.SchemaField("referrer","STRING"),
        bigquery.SchemaField("user_agent","STRING"),
        bigquery.SchemaField("mcc","STRING"),
        bigquery.SchemaField("mnc","STRING"),
        bigquery.SchemaField("ip_address","STRING"),
        bigquery.SchemaField("isp","STRING"),
        bigquery.SchemaField("region","STRING"),
        bigquery.SchemaField("country","STRING"),
        bigquery.SchemaField("country_subdivision","STRING"),
        bigquery.SchemaField("city","STRING"),
        bigquery.SchemaField("postal_code","STRING"),
        bigquery.SchemaField("language","STRING"),
        bigquery.SchemaField("device_name","STRING"),
        bigquery.SchemaField("device_type","STRING"),
        bigquery.SchemaField("os_name","STRING"),
        bigquery.SchemaField("platform","STRING"),
        bigquery.SchemaField("api_level","STRING"),
        bigquery.SchemaField("sdk_version","STRING"),
        bigquery.SchemaField("os_version","STRING"),
        bigquery.SchemaField("random","STRING"),
        bigquery.SchemaField("nonce","STRING"),
        bigquery.SchemaField("random_user_id","STRING"),
        bigquery.SchemaField("environment","STRING"),
        bigquery.SchemaField("tracking_enabled","STRING"),
        bigquery.SchemaField("tracking_limited","STRING"),
        bigquery.SchemaField("att_status","STRING"),
        bigquery.SchemaField("third_party_sharing_disabled","STRING"),
        bigquery.SchemaField("timezone","STRING"),
        bigquery.SchemaField("event","STRING"),
        bigquery.SchemaField("event_name","STRING"),
        bigquery.SchemaField("last_time_spent","STRING"),
        bigquery.SchemaField("time_spent","STRING"),
        bigquery.SchemaField("session_count","STRING"),
        bigquery.SchemaField("lifetime_session_count","STRING"),
        bigquery.SchemaField("is_reattributed","STRING"),
        bigquery.SchemaField("is_imported","STRING"),
        bigquery.SchemaField("deeplink","STRING"),
        bigquery.SchemaField("partner_parameters","STRING"),
        bigquery.SchemaField("revenue_float","STRING"),
        bigquery.SchemaField("revenue","STRING"),
        bigquery.SchemaField("currency","STRING"),
        bigquery.SchemaField("revenue_usd","STRING"),
        bigquery.SchemaField("revenue_usd_cents","STRING"),
        bigquery.SchemaField("reporting_revenue","STRING"),
        bigquery.SchemaField("reporting_currency","STRING"),
        bigquery.SchemaField("cost_type","STRING"),
        bigquery.SchemaField("cost_amount","STRING"),
        bigquery.SchemaField("cost_currency","STRING"),
        bigquery.SchemaField("reporting_cost","STRING"),
        bigquery.SchemaField("cost_id_md5","STRING"),
        bigquery.SchemaField("push_token","STRING"),
        bigquery.SchemaField("sk_ts","STRING"),
        bigquery.SchemaField("sk_payload","STRING"),
        bigquery.SchemaField("sk_version","STRING"),
        bigquery.SchemaField("sk_network_id","STRING"),
        bigquery.SchemaField("sk_campaign_id","STRING"),
        bigquery.SchemaField("sk_transaction_id","STRING"),
        bigquery.SchemaField("sk_app_id","STRING"),
        bigquery.SchemaField("sk_attribution_signature","STRING"),
        bigquery.SchemaField("sk_invalid_signature","STRING"),
        bigquery.SchemaField("sk_redownload","STRING"),
        bigquery.SchemaField("sk_fidelity_type","STRING"),
        bigquery.SchemaField("sk_source_app_id","STRING"),
        bigquery.SchemaField("sk_conversion_value","STRING"),
        bigquery.SchemaField("sk_did_win","STRING"),
        bigquery.SchemaField("partner","STRING"),
        bigquery.SchemaField("reporting_revenue_min","STRING"),
        bigquery.SchemaField("reporting_revenue_max","STRING"),
        bigquery.SchemaField("app_token","STRING"),
        bigquery.SchemaField("publisher_parameters","STRING"),
        bigquery.SchemaField("label","STRING"),
        bigquery.SchemaField("secret_id","STRING"),
        bigquery.SchemaField("gclid","STRING"),
        bigquery.SchemaField("google_ads_campaign_type","STRING"),
        bigquery.SchemaField("google_ads_campaign_name","STRING"),
        bigquery.SchemaField("google_ads_campaign_id","STRING"),
        bigquery.SchemaField("google_ads_adgroup_name","STRING"),
        bigquery.SchemaField("google_ads_adgroup_id","STRING"),
        bigquery.SchemaField("google_ads_creative_id","STRING"),
        bigquery.SchemaField("google_ads_network_type","STRING"),
        bigquery.SchemaField("google_ads_network_subtype","STRING"),
        bigquery.SchemaField("google_ads_keyword","STRING"),
        bigquery.SchemaField("google_ads_matchtype","STRING"),
        bigquery.SchemaField("google_ads_placement","STRING"),
        bigquery.SchemaField("google_ads_video_id","STRING"),
        bigquery.SchemaField("google_ads_external_customer_id","STRING"),
        bigquery.SchemaField("google_ads_ad_type","STRING"),
        bigquery.SchemaField("search_term","STRING"),
        bigquery.SchemaField("fb_campaign_group_name","STRING"),
        bigquery.SchemaField("fb_campaign_group_id","STRING"),
        bigquery.SchemaField("fb_campaign_name","STRING"),
        bigquery.SchemaField("fb_campaign_id","STRING"),
        bigquery.SchemaField("fb_adgroup_name","STRING"),
        bigquery.SchemaField("fb_adgroup_id","STRING"),
        bigquery.SchemaField("fb_ad_objective_name","STRING"),
        bigquery.SchemaField("fb_account_id","STRING"),
        bigquery.SchemaField("fb_platform_position","STRING"),
        bigquery.SchemaField("fb_is_aaa_campaign","STRING"),
        bigquery.SchemaField("fb_creative_id","STRING"),
        bigquery.SchemaField("tweet_id","STRING"),
        bigquery.SchemaField("twitter_line_item_id","STRING"),
        bigquery.SchemaField("iad_creative_set_name","STRING"),
        bigquery.SchemaField("iad_creative_set_id","STRING"),
        bigquery.SchemaField("iad_conversion_type","STRING"),
        bigquery.SchemaField("iad_keyword_matchtype","STRING"),
    ],
    skip_leading_rows=1,
    # The source format defaults to CSV, so the line below is optional.
    source_format=bigquery.SourceFormat.CSV,
)

destination_table = client.get_table(table_id)  # Make an API request.

In [18]:
# 9) Insert all the files that the string is not file_not_import and are a valid .csv.gz file
for file in files:
    if(str(file).find(file_not_import) == -1 and str(file).find('.csv.gz') != -1):
        # location file to import
        uri = "gs://tgm-adjust/" + str(file.name)
        # job to import
        load_job = client.load_table_from_uri(
            uri, table_id, job_config=job_config
        )
        # execute the job and import the file
        load_job.result()
        print("Loaded {} rows.".format(destination_table.num_rows))
        # 9.1) After Import move the file to Imported folder
        move_blob('tgm-adjust', str(file.name), 'tgm_adjust_bq', str(file.name))